In [1]:
import pandas as pd
import os
import numpy as np

from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.func_demograficas import *

In [ ]:
def codif_diada_depto(df):
    "Codifica díadas por departamento"
    cod_depto_destino_str = df['depto_destino'].astype(str).str.zfill(2)
    cod_diada_depto = (df['depto_origen'].astype(str) + cod_depto_destino_str).astype(int)
    return cod_diada_depto

In [6]:
mig = cargar_migrantes_internos()

mig.columns

Index(['depto_origen', 'depto_destino', 'LOC', 'SECC', 'SEGM', 'VIVID',
       'TIPO_VIVIE', 'HOGCOD', 'HOGID', 'PERPH02', 'PERPA01', 'PERER02',
       'PERNA01', 'PERNA02', 'PERMI01', 'PERMI01_1', 'PERMI01_2', 'PERMI01_3',
       'PERMI01_4', 'PERMI02', 'PERMI02_1', 'PERMI05', 'PERMI05_1', 'PERMI06',
       'PERMI06_1', 'PERMI06_2', 'PERMI06_3', 'PERMI06_4', 'PERMI07',
       'PERMI07_1', 'PERMI07_2', 'PERMI07_3', 'PERMI07_4', 'PERED00',
       'PERED01', 'PERED02', 'PERED02_1', 'PERED02_2', 'PERED02_3',
       'PERED02_4', 'PERED03_R', 'PERED03_1', 'PERED03_2', 'PERED04_R',
       'PERED05_R', 'PERED06_R', 'CODIGO_CAR', 'PERED08', 'NIVELEDU_R',
       'loc_origen', 'loc_destino'],
      dtype='object')

In [13]:
mig['cod'] = codif_diada_depto(mig)

In [14]:
mig.head()

,depto_origen,depto_destino,LOC,SECC,SEGM,VIVID,TIPO_VIVIE,HOGCOD,HOGID,PERPH02,...,PERED04_R,PERED05_R,PERED06_R,CODIGO_CAR,PERED08,NIVELEDU_R,loc_origen,loc_destino,tramo,cod
0,3,1,20,1,1,5,1,102001001000501,1,1,...,2,3,4,55131,1,9,3732,1020,20-24,301
1,17,1,20,1,1,30,1,102001001003001,1,2,...,0,2,4,55152,1,9,17220,1020,20-24,1701
2,18,1,20,1,1,36,1,102001001003601,1,1,...,2,2,0,33101,1,5,18220,1020,15-19,1801
3,2,1,20,1,1,46,1,102001001004601,1,1,...,2,1,0,33101,1,5,2220,1020,20-24,201
4,12,1,20,1,1,93,1,102001001009301,1,2,...,2,1,0,22101,1,4,12320,1020,25-29,1201


In [18]:
n = mig.groupby('cod').size()
n[0:5]

cod
102      914
103    33127
104     1387
105     2100
106      982
dtype: int64

In [40]:
sexo = [mig.loc[mig.PERPH02==i].groupby('cod').size() for i in [1, 2]]
sexo[0]

cod
102       428
103     16049
104       684
105       944
106       454
        ...  
1914       84
1915        6
1916       12
1917       13
1918       24
Length: 342, dtype: int64

In [50]:
mig_data = mig.groupby(['depto_origen', 'depto_destino', 'cod']) \
  .apply(lambda x: pd.Series({
      'n':             x['PERPH02'].count(),
      'edad_promedio': x['PERNA01'].mean(),
      'edad_mediana':  x['PERNA01'].median(),
      'varones':  x['PERPH02'].loc[x['PERPH02']==1].count(),
      'mujeres':  x['PERPH02'].loc[x['PERPH02']==2].count(),
  })
).reset_index()

# cambia a integers
cols = ['n', 'edad_mediana', 'varones', 'mujeres']
mig_data.loc[:, cols] = mig_data.loc[:, cols].astype(int)

# agrega índice de masculinidad
im = (mig_data.loc[:, 'varones'] / mig_data.loc[:, 'mujeres'])

mig_data['ind_masc'] = round(im*100, 2)

mig_data.head()

,depto_origen,depto_destino,cod,n,edad_promedio,edad_mediana,varones,mujeres,ind_masc
0,1,2,102,914,30.855580,28,428,486,88.07
1,1,3,103,33127,35.474749,33,16049,17078,93.97
2,1,4,104,1387,32.630137,30,684,703,97.30
3,1,5,105,2100,34.265238,32,944,1156,81.66
4,1,6,106,982,31.549898,30,454,528,85.98


In [51]:
mig_data.to_csv('tablas/datos_tablero.csv', index=False)

In [54]:
os.chdir('/home/guillermo/Documentos/GitHub/migraciones_internas/data/')
mig_data.to_csv('datos_tablero.csv', index=False)

In [11]:

agrupar_por_tramos(mig, 'tramo', 'PERNA01', 'PERPH02')

,sexo,tramo,personas,porc_pers,sexo_label
0,1,+95,-21,-0.014117,varones
1,1,10-14,-5234,-3.518443,varones
2,1,15-19,-6785,-4.561069,varones
3,1,20-24,-11190,-7.522234,varones
4,1,25-29,-8301,-5.580167,varones
5,1,30-34,-7482,-5.029612,varones
6,1,35-39,-6184,-4.157059,varones
7,1,40-44,-4403,-2.959821,varones
8,1,45-49,-3659,-2.459683,varones
9,1,5-9,-6188,-4.159748,varones
